In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dropout, Dense, LSTM, Input, Add
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import plot_model

In [ ]:
data=pd.read_csv('../input/crude-oil-prices/Train.csv',delimiter=';')

In [ ]:
data.Date=pd.to_datetime(data.Date)

# instead of using to datetime 

def month_founder(x):

    if x[:3]=='Jul':
    
        return int(7)
        
    elif x[:3]=='Jun':
    
        return int(6)
        
    elif x[:3]=='May':
    
        return int(5)
        
    elif x[:3]=='May':
    
        return int(5)
        
    elif x[:3]=='Apr':
    
        return int(4)
        
    elif x[:3]=='Mar':
    
        return int(3)
        
    elif x[:3]=='Feb':
    
        return int(2)
        
    elif x[:3]=='Jan':
    
        return int(1)

In [ ]:
data=data.replace('-', np.nan)

In [ ]:
data=data.dropna()

In [ ]:
data=data.sort_values(by='Date')

In [ ]:
data.head()

In [ ]:
plt.plot(data.Date[0:10],data.Open[0:10], color='r')
plt.xticks(rotation=90)
plt.xlabel('$Date$')
plt.ylabel('Price in US Dollars')
plt.grid()
plt.show()

In [ ]:
data_prices=data.Open.values

In [ ]:
print('Shape of data_prices: ',data_prices.shape)

In [ ]:
data_prices=data_prices.reshape(data_prices.shape[0],1)

In [ ]:
scaler=MinMaxScaler(feature_range=(0, 1))
s_data_prices=scaler.fit_transform(data_prices)

In [ ]:
def train_test_generator(data):
    X_data=[]
    y_data=[]
    batch_size=10
    for price in range (len(data)-batch_size-1):
        X_data.append(data[price:price+batch_size])
        y_data.append(data[price+batch_size+1])
    X_data=np.array(X_data).reshape(np.array(X_data).shape[0],1,np.array(X_data).shape[1])

    y_data=np.array(y_data)
    return X_data, y_data

In [ ]:
len(s_data_prices)/2

In [ ]:
X_train, y_train=train_test_generator(s_data_prices[:571])

In [ ]:
X_test, y_test=train_test_generator(s_data_prices[571:])

In [ ]:
print('Shape of X_train: ',X_train.shape)
print('Shape of y_train: ',y_train.shape)
print('Shape of X_test: ',X_test.shape)
print('Shape of y_test: ',y_test.shape)

In [ ]:
Input_layer=Input(shape=X_train.shape[1:])
layer0=LSTM(128,return_sequences=True)(Input_layer)
layer1=Dropout(0.5)(layer0)
layer2_1=LSTM(64,return_sequences=True)(layer1)
layer2_2=LSTM(64,return_sequences=True)(layer1)
layer3=Add()([layer2_1,layer2_2])
layer4=LSTM(32)(layer3)
layer5=Dropout(0.5)(layer4)
Output_layer=Dense(1)(layer5)
model=Model(inputs=Input_layer,
           outputs=Output_layer)
model.compile(optimizer='adam',
             loss='mean_squared_error',
             metrics=['accuracy'])

In [ ]:
plot_model(model,
          show_shapes=True,)

In [ ]:
history=model.fit(X_train,y_train,
         batch_size=16,
         validation_split=0.25,
         epochs=50,
         validation_data=(X_test,y_test))

In [ ]:
history_df=pd.DataFrame(history.history)
history_df.head()

In [ ]:
history_df[['loss','val_loss']].plot()

In [ ]:
history_df[['accuracy','val_accuracy']].plot()

In [ ]:
train_preds=model.predict(X_train)

In [ ]:
test_preds=model.predict(X_test)

In [ ]:
preds_r=scaler.inverse_transform(test_preds)
y_test_r=scaler.inverse_transform(y_test)

In [ ]:
plt.figure()
plt.plot(preds_r, label='predictions')
plt.plot(y_test_r,label='real')
plt.grid()
plt.show()